In [13]:
import pandas as pd
import numpy as np
filename = 'diabetes.csv'
df = pd.read_csv(filename)
df = df.astype(float)
train = df.sample(frac=0.8,random_state=100)
test = df.drop(train.index)
print(test)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6.0    148.0           72.0           35.0      0.0  33.6   
2            8.0    183.0           64.0            0.0      0.0  23.3   
4            0.0    137.0           40.0           35.0    168.0  43.1   
13           1.0    189.0           60.0           23.0    846.0  30.1   
14           5.0    166.0           72.0           19.0    175.0  25.8   
..           ...      ...            ...            ...      ...   ...   
733          2.0    106.0           56.0           27.0    165.0  29.0   
738          2.0     99.0           60.0           17.0    160.0  36.6   
752          3.0    108.0           62.0           24.0      0.0  26.0   
763         10.0    101.0           76.0           48.0    180.0  32.9   
764          2.0    122.0           70.0           27.0      0.0  36.8   

     DiabetesPedigreeFunction   Age  Outcome  
0                       0.627  50.0      1.0  
2                

In [4]:
outcome_group = train.groupby(df.columns[-1])
n_attr = len(df.columns)-1
summaries = {}
for classValue,instances in outcome_group:
  attr_mv = []
  mean= list(instances.mean(axis=0).values)
  stdev= list(instances.std(axis=0).values)
  
  for i in range (n_attr):
    attr_mv.append([mean[i],stdev[i]])
  summaries[classValue]=attr_mv
  print(classValue)
  print(summaries)

0.0
{0.0: [[3.2130325814536342, 2.972610408317797], [109.76942355889724, 26.905603970990228], [68.4812030075188, 17.522677985264334], [20.100250626566417, 14.665790191703916], [72.43358395989975, 96.65287822555396], [30.435338345864665, 7.7580581756384905], [0.43053634085213033, 0.2915681446463359], [30.86967418546366, 11.496036356224192]]}
1.0
{0.0: [[3.2130325814536342, 2.972610408317797], [109.76942355889724, 26.905603970990228], [68.4812030075188, 17.522677985264334], [20.100250626566417, 14.665790191703916], [72.43358395989975, 96.65287822555396], [30.435338345864665, 7.7580581756384905], [0.43053634085213033, 0.2915681446463359], [30.86967418546366, 11.496036356224192]], 1.0: [[4.902325581395349, 3.852993033222635], [142.09302325581396, 29.85260829685783], [71.14418604651163, 21.708619485476845], [22.29767441860465, 18.08223024522044], [89.81860465116279, 114.64875097814037], [35.415813953488374, 7.527291849134547], [0.5482744186046512, 0.3672769639764627], [36.948837209302326, 1

In [5]:
import math
def calculateProb(x,mean,stdev):
  exponent = math.exp(-math.pow(x-mean,2)/(2*(math.pow(stdev,2))))
  return( 1/(math.sqrt(2*math.pi)*math.pow(stdev,2)))*exponent

def calculateClassProb(summaries,x_vec):
  probablities = {}
  for classValue,classSummaries in summaries.items():
    probablities[classValue]=1
    for i in range(len(classSummaries)):
      mean,stdev = classSummaries[i]
      x = x_vec[i]
      probablities[classValue]*= calculateProb(x,mean,stdev)
  return probablities

def predict(summaries,x_vec):
  prob = calculateClassProb(summaries,x_vec)
  bestlabel,bestprob = None,-1
  for classValue,probality in prob.items():
    if bestlabel is None or probality>bestprob:
      bestlabel = classValue
      bestprob = probality
  return bestlabel


In [12]:
predictions=[]
testset = test.values.tolist()

for i in range(len(testset)):
  result = predict(summaries, testset[i])
  predictions.append(result)


In [11]:
def getaccuracy(test,predictions):
  correct = 0
  for i in range(len(test)):
    if test.iloc[i,-1]==predictions[i]:
      correct+= 1
  return (correct/float(len(test)))*100.0
accuracy = getaccuracy(test,predictions)
print(f'Accuracy: {accuracy}')
print(f'Split {len(df)} rows into train {len(train)} and test {len(test)}') 

Accuracy: 74.67532467532467
Split 768 rows into train 614 and test 154
